## Querying simbad
The function 'simbad' takes as input the name of a star, the SWEET-Cat table and a TAP service for simbad. It certifies the star inputted is among those in SWEET-Cat and creates for this star a text file with 4 columns corresponding, in this order, to the value of vsini, the upper bound, the lower bound, the bibcode and the catalog name ('simbad').

In [1]:
# imports

# SWEET-Cat
import urllib
import pandas as pd
import numpy as np
from astropy import coordinates as coord
from astropy import units as u

from astropy.table import Table
from astropy.io.ascii import convert_numpy

# astroquery
from astroquery.simbad import Simbad
from astroquery.utils.tap.core import TapPlus #tap service

In [4]:
#Download SWEET-Cat
sweetCat_table_url = "https://sweetcat.iastro.pt/catalog/SWEETCAT_Dataframe.csv"
converters={'gaia_dr2': [convert_numpy(np.int64)],'gaia_dr3': [convert_numpy(np.int64)] }
sweet_table = Table.read(sweetCat_table_url, encoding='UTF-8',format='csv', converters=converters)

sweet_stars = sweet_table['Name']

simbadtap = TapPlus(url="http://simbad.u-strasbg.fr/simbad/sim-tap")

def simbad(star_name, sweet_table, simbadtap): #query simbad catalog and make csv file for every star
    
    sweet_stars = sweet_table['Name']
    
    # check if star is in SWEET-Cat
    if star_name not in sweet_stars:
        return "Not a SWEET-Cat star name."
    
    # Get star id
    gaia2_id = sweet_table[sweet_table['Name'] == star_name]['gaia_dr2']
    gaia3_id = sweet_table[sweet_table['Name'] == star_name]['gaia_dr3']
    
    star_id2 = "gaia_dr2 " + str(gaia2_id[0])
    star_id3 = "gaia_dr3 " + str(gaia3_id[0])
    
    # Query Simbad and make dataframe
    #simbadtap = TapPlus(url="http://simbad.u-strasbg.fr/simbad/sim-tap")
    
    job = simbadtap.launch_job("select main_id,mrot.vsini,mrot.vsini_err, mrot.bibcode from basic join ident as i on i.oidref = oid join mesRot as mrot on mrot.oidref = oid where i.id in ('"+ star_id2 + "')")
    
    result = job.get_results()

    cat = ['simbad']*len(result)
    
    df = result.to_pandas()
    df = df.drop(['main_id'], axis=1)
    #print(df['vsini_err'])
    err_low = -df['vsini_err']  
    #df['vsini_err2'] = err_low
    df.insert(2, 'vsini_err2', err_low)
    df['Catalogue'] = cat
    
    # Make csv file with the values
    if "/" in star_name or "*" in star_name:
        file_name = star_id2
    else:
        file_name = star_name
        
    df.to_csv('{}.txt'.format(file_name), index=False, header=False, sep = '\t')
    
    return len(result)

In [18]:
count_tot = 0
count_0 = 0
for star in sweet_stars:
    print(star)
    res = simbad(star, sweet_table, simbadtap)
    print(res)
    count_tot += 1
    if res == 0:
        count_0 += 1

11 Com
<Table length=3>
   name    dtype   unit           description          n_bad
--------- ------- ------ ----------------------------- -----
  main_id  object        Main identifier for an object     0
    vsini float32 km / s                        V sini     0
vsini_err float32                                error     2
  bibcode  object                  measurement bibcode     0
Jobid: None
Phase: COMPLETED
Owner: None
Output file: sync_20230322191859.xml
Results: None
 main_id  vsini  vsini_err       bibcode      
          km / s                              
--------- ------ --------- -------------------
*  11 Com   10.0        -- 1970CoAsi.239....1B
*  11 Com   19.0        -- 1970CoKwa.189....0U
*  11 Com   2.96      1.15 2015A&A...574A..50J
0     NaN
1     NaN
2    1.15
Name: vsini_err, dtype: float32
3
11 UMi
<Table length=2>
   name    dtype   unit           description          n_bad
--------- ------- ------ ----------------------------- -----
  main_id  object        M

<Table length=3>
   name    dtype   unit           description          n_bad
--------- ------- ------ ----------------------------- -----
  main_id  object        Main identifier for an object     0
    vsini float32 km / s                        V sini     0
vsini_err float32                                error     2
  bibcode  object                  measurement bibcode     0
Jobid: None
Phase: COMPLETED
Owner: None
Output file: sync_20230322191901.xml
Results: None
  main_id   vsini  vsini_err       bibcode      
            km / s                              
----------- ------ --------- -------------------
* pi.02 UMa    8.0        -- 1970CoAsi.239....1B
* pi.02 UMa   17.0        -- 1970CoKwa.189....0U
* pi.02 UMa   2.08      0.48 2015A&A...574A..50J
0     NaN
1     NaN
2    0.48
Name: vsini_err, dtype: float32
3
42 Dra
<Table length=3>
   name    dtype   unit           description          n_bad
--------- ------- ------ ----------------------------- -----
  main_id  object    

75 Cet
<Table length=4>
   name    dtype   unit           description          n_bad
--------- ------- ------ ----------------------------- -----
  main_id  object        Main identifier for an object     0
    vsini float32 km / s                        V sini     0
vsini_err float32                                error     2
  bibcode  object                  measurement bibcode     0
Jobid: None
Phase: COMPLETED
Owner: None
Output file: sync_20230322191903.xml
Results: None
 main_id  vsini  vsini_err       bibcode      
          km / s                              
--------- ------ --------- -------------------
*  75 Cet  3.236     0.504 2021A&A...647A.157S
*  75 Cet   0.83      0.45 2015A&A...574A..50J
*  75 Cet   1.22        -- 2011A&A...536A..71J
*  75 Cet    0.0        -- 2008AJ....135..209M
0    0.504
1    0.450
2      NaN
3      NaN
Name: vsini_err, dtype: float32
4
8 Umi
<Table length=0>
   name    dtype   unit           description         
--------- ------- ------ --------

0
BD+20 2457
<Table length=0>
   name    dtype   unit           description         
--------- ------- ------ -----------------------------
  main_id  object        Main identifier for an object
    vsini float32 km / s                        V sini
vsini_err float32                                error
  bibcode  object                  measurement bibcode
Jobid: None
Phase: COMPLETED
Owner: None
Output file: sync_20230322191906.xml
Results: None
main_id vsini  vsini_err bibcode
        km / s                  
------- ------ --------- -------
Series([], Name: vsini_err, dtype: float32)
0
BD+20 274
<Table length=0>
   name    dtype   unit           description         
--------- ------- ------ -----------------------------
  main_id  object        Main identifier for an object
    vsini float32 km / s                        V sini
vsini_err float32                                error
  bibcode  object                  measurement bibcode
Jobid: None
Phase: COMPLETED
Owner: None
Outpu

<Table length=0>
   name    dtype   unit           description         
--------- ------- ------ -----------------------------
  main_id  object        Main identifier for an object
    vsini float32 km / s                        V sini
vsini_err float32                                error
  bibcode  object                  measurement bibcode
Jobid: None
Phase: COMPLETED
Owner: None
Output file: sync_20230322191908.xml
Results: None
main_id vsini  vsini_err bibcode
        km / s                  
------- ------ --------- -------
Series([], Name: vsini_err, dtype: float32)
0
CoRoT-13
<Table length=0>
   name    dtype   unit           description         
--------- ------- ------ -----------------------------
  main_id  object        Main identifier for an object
    vsini float32 km / s                        V sini
vsini_err float32                                error
  bibcode  object                  measurement bibcode
Jobid: None
Phase: COMPLETED
Owner: None
Output file: sync_2

...

<Table length=0>
   name    dtype   unit           description         
--------- ------- ------ -----------------------------
  main_id  object        Main identifier for an object
    vsini float32 km / s                        V sini
vsini_err float32                                error
  bibcode  object                  measurement bibcode
Jobid: None
Phase: COMPLETED
Owner: None
Output file: sync_20230322193012.xml
Results: None
main_id vsini  vsini_err bibcode
        km / s                  
------- ------ --------- -------
Series([], Name: vsini_err, dtype: float32)
0
TOI-2207
<Table length=0>
   name    dtype   unit           description         
--------- ------- ------ -----------------------------
  main_id  object        Main identifier for an object
    vsini float32 km / s                        V sini
vsini_err float32                                error
  bibcode  object                  measurement bibcode
Jobid: None
Phase: COMPLETED
Owner: None
Output file: sync_2

   name    dtype   unit           description         
--------- ------- ------ -----------------------------
  main_id  object        Main identifier for an object
    vsini float32 km / s                        V sini
vsini_err float32                                error
  bibcode  object                  measurement bibcode
Jobid: None
Phase: COMPLETED
Owner: None
Output file: sync_20230322193015.xml
Results: None
main_id vsini  vsini_err bibcode
        km / s                  
------- ------ --------- -------
Series([], Name: vsini_err, dtype: float32)
0
TOI-620
<Table length=0>
   name    dtype   unit           description         
--------- ------- ------ -----------------------------
  main_id  object        Main identifier for an object
    vsini float32 km / s                        V sini
vsini_err float32                                error
  bibcode  object                  measurement bibcode
Jobid: None
Phase: COMPLETED
Owner: None
Output file: sync_20230322193015.xml


   name    dtype   unit           description          n_bad
--------- ------- ------ ----------------------------- -----
  main_id  object        Main identifier for an object     0
    vsini float32 km / s                        V sini     0
vsini_err float32                                error     1
  bibcode  object                  measurement bibcode     0
Jobid: None
Phase: COMPLETED
Owner: None
Output file: sync_20230322193017.xml
Results: None
    main_id     vsini  vsini_err       bibcode      
                km / s                              
--------------- ------ --------- -------------------
UCAC2  35929957    3.2        -- 2018AJ....155...21P
0   NaN
Name: vsini_err, dtype: float32
1
K2-372
<Table length=2>
   name    dtype   unit           description          n_bad
--------- ------- ------ ----------------------------- -----
  main_id  object        Main identifier for an object     0
    vsini float32 km / s                        V sini     0
vsini_err float32   

<Table length=0>
   name    dtype   unit           description         
--------- ------- ------ -----------------------------
  main_id  object        Main identifier for an object
    vsini float32 km / s                        V sini
vsini_err float32                                error
  bibcode  object                  measurement bibcode
Jobid: None
Phase: COMPLETED
Owner: None
Output file: sync_20230322193020.xml
Results: None
main_id vsini  vsini_err bibcode
        km / s                  
------- ------ --------- -------
Series([], Name: vsini_err, dtype: float32)
0
K2-386
<Table length=1>
   name    dtype   unit           description          n_bad
--------- ------- ------ ----------------------------- -----
  main_id  object        Main identifier for an object     0
    vsini float32 km / s                        V sini     0
vsini_err float32                                error     1
  bibcode  object                  measurement bibcode     0
Jobid: None
Phase: COMPLET

<Table length=0>
   name    dtype   unit           description         
--------- ------- ------ -----------------------------
  main_id  object        Main identifier for an object
    vsini float32 km / s                        V sini
vsini_err float32                                error
  bibcode  object                  measurement bibcode
Jobid: None
Phase: COMPLETED
Owner: None
Output file: sync_20230322193022.xml
Results: None
main_id vsini  vsini_err bibcode
        km / s                  
------- ------ --------- -------
Series([], Name: vsini_err, dtype: float32)
0
K2-401
<Table length=0>
   name    dtype   unit           description         
--------- ------- ------ -----------------------------
  main_id  object        Main identifier for an object
    vsini float32 km / s                        V sini
vsini_err float32                                error
  bibcode  object                  measurement bibcode
Jobid: None
Phase: COMPLETED
Owner: None
Output file: sync_202

In [19]:
print(count_tot)
print(count_0) #how many stars have results for vsini in simbad

3803
1844
